# Imports and Constants

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from neural_networks.layers import *
from neural_networks.networks import *
from math_util.vector_functions import sum_of_squares_cost

In [ ]:
LEARNING_RATE_2D = 1e-3
SAMPLE_SIZE_2D = 5
ITERATION_COUNT_2D = 5_000
ORDER_2D = 1_000

# 2D Example

This example is meant to take an image where white pixels represent the edge of a surface and try to find an implicit function for the surface as described below.

## Network Creation

In [ ]:
nn_2d = Network([
    PolynomialLayer(
        2, 1,
        learning_rate=LEARNING_RATE_2D,
        order=ORDER_2D,
    ),
])

## Setting up the Input

I'll use the following image as the basis for the implicit function.

![2D Figure](2dfigure.bmp)

The red region is the inside region, the blue is the outside region and the white surface is the surface that I want to make an implicit function for.

Therefore, I need the network to represent a function, $f: \mathbb R^2 \rightarrow \mathbb R$, such that $f(x,y)=0$ on the surface, $f(x,y)<0$ within the surface and $f(x,y)>0$ outside the surface.

I'll train the network for the $f(x,y)=0$ on the surface part but the sign on either side isn't important as I can just take the negative of the function if it's the wrong way round.

In [ ]:
_img_pil = Image.open("2dfigure.bmp")
img = np.array(_img_pil)
del _img_pil

img_height, img_width = img.shape[0], img.shape[1]

print(img.shape)
plt.imshow(img)

I need a matrix of the coordinates of each point on the image. I'll normalise these so that they work better in the neural network.

In [ ]:
coords = np.dstack(np.meshgrid(
    np.linspace(0, 1, img.shape[1]),
    np.linspace(0, 1, img.shape[0]),
)).transpose((1,0,2))

plt.imshow(np.sum(coords, axis=2), cmap="gray")

Next, I find the coordinates of the points on the surface of the shape.

In [ ]:
img_surface_mask = (img[:,:,0] == 255) & (img[:,:,1] == 255) & (img[:,:,1] == 255)
plt.imshow(np.where(
    img_surface_mask[:,:,np.newaxis],
    np.array([ 255, 255, 255 ]),
    np.array([ 0, 0, 0 ]),
))

img_surface_poss = coords[img_surface_mask]
print(img_surface_poss)

Now that I have the coordinates for the surface, I'll take a selection of them to model a point cloud.

I won't do this randomly for the following reasons:
1. The same results can be reproduced on each run of the code
2. The points should be quite evenly distributed

In [ ]:
img_point_cloud = img_surface_poss[np.arange(0, img_surface_poss.shape[0], 2)]
img_point_cloud.shape[0]

## Training the Network

Now, I can train the network.

During the training process, I'll output the results at different stages to see how long it takes for progress to be made and at what points progress isn't being made.

In [ ]:
def nn_2d_outputs_to_img(nn_2d_outputs):
    return np.where(
        np.isclose(nn_2d_outputs, 0, atol=1e-3),
        np.array([ 1.0, 1.0, 1.0 ], dtype=np.float64),
        np.where(
            nn_2d_outputs < 0,
            np.array([ 1.0, 0, 0 ], dtype=np.float64) * np.tanh(np.power(np.abs(nn_2d_outputs), 0.4)),
            np.array([ 0, 0, 1.0 ], dtype=np.float64) * np.tanh(np.power(np.abs(nn_2d_outputs), 0.4)),
        ),
    )

learn_X = img_point_cloud
learn_Y = np.zeros(shape=(img_point_cloud.shape[0],1))

checkpoint_count = 10
checkpoints: List = np.linspace(0, ITERATION_COUNT_2D, checkpoint_count, dtype=int).tolist()

fig,axes = plt.subplots(checkpoint_count+1,1)
fig.set_figwidth(2)
fig.set_figheight(2*checkpoint_count)

for i, _ in enumerate(nn_2d.learn_stochastic_it(
    xs=learn_X,
    exps=learn_Y,
    cost_func=sum_of_squares_cost,
    sample_size=SAMPLE_SIZE_2D,
    iteration_count=ITERATION_COUNT_2D,
    provide_cost_output=False,
)):

    if i in checkpoints:
        checkpoint_idx = checkpoints.index(i)
        pred_X = coords.reshape((img_height*img_width, 2))
        nn_2d_outputs = nn_2d.forwards_multi(pred_X).reshape((img_height,img_width,1))
        axes[checkpoint_idx].imshow(nn_2d_outputs_to_img(nn_2d_outputs))
        axes[checkpoint_idx].set_title(f"Iteration {i}")

axes[-1].imshow(img)
axes[-1].set_title("Reference Image")

print("Average cost after training: " + str(np.mean(sum_of_squares_cost.f_multi(
    nn_2d.forwards_multi(learn_X),
    learn_Y
))))

fig.tight_layout()
plt.show()